<a href="https://colab.research.google.com/github/ArtoAnnila/GalaxyDynamiX/blob/main/GalaxyDynamiX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GalaxyDynamiX**

* Run cells to get figures.
* Edit code to change parameters and profiles.



In [ ]:
#@title MOUNT
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
#@title VELOCITY DISPERSIONS AND ROTATION CURVES

# GalaxyCurveX: Computes galaxy velocity dispersion and rotation curves
# including the acceleration effect of cosmic expansion.

import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gammaincinv
from scipy.optimize import minimize_scalar
from scipy.integrate import quad

path="/content/gdrive/MyDrive/ColabMaps/"

plt.rcParams['font.family'] = 'DejaVu Serif'

# ---------------- Constants ---------------------------------------------------
G = 6.67430e-11             # Gravitational constant, m^3 kg^-1 s^-2
c = 299792458               # Speed of light, m/s
M_sun = 1.98847e30          # Solar mass, kg
ly_in_m = 9.461e15          # Light-year in meters
pc_in_m = 3.0857e16         # Parsec in meters
ly_in_pc = 0.306601         # Light-year in parsecs
pc_in_ly = 3.26156          # Parsec in light-years
t_universe = 13.8e9 * 365.25 * 24 * 3600  # Age of universe in seconds
R = c*t_universe            # Causal radius
a_R = c / t_universe        # Cosmic acceleration
rho = 1/(4*math.pi*G*t_universe**2)       # universal density kgm^-3
rho_AU = rho/M_sun*(pc_in_m)**3
M_env = 1e15 * M_sun        # A parameter to model environment

# ---------------- Galaxy Parameters -------------------------------------------
M_total = 1e11 * M_sun      # General object baryonic mass
M_cluster = 1e15 * M_sun    # Cluster
M_galaxy = 1e11 * M_sun     # Galaxy
M_dwarf = 1e7 * M_sun       # Dwarf

# Hernquist model (Cluster, Elliptical Galaxy, Spheroid Dwarf)
r_scale_ly = 10000          # General object scale radius
r_scale_m = r_scale_ly * ly_in_m

r_cluster = 1e6             # Cluster
r_galaxy = 10000            # Galaxy
r_dwarf = 1000              # Dwarf
r_cluster_m = r_cluster * ly_in_m         # Cluster
r_galaxy_m = r_galaxy * ly_in_m           # Galaxy
r_dwarf_m = r_dwarf * ly_in_m             # Dwarf

# Sérsic AU (Spiral Galaxy)
M_spiral_solar = M_total/M_sun
M_spiral_solar = 1e11       # Spiral mass in Solar masses (M_sun)
R_e_ly = 12000
#R_e_ly = 1000              # Dwarf
R_e_m = R_e_ly * ly_in_m
R_e_pc = R_e_ly * ly_in_m / pc_in_m
n_ser = 1.0
bn = gammaincinv(2 * n_ser, 0.5)
pn = 1.0 - 0.6097 / n_ser + 0.05463 / n_ser**2
mass_to_light_ratio_ser = 1.0

G_AU = 4.302e-3             # G in units of pc * (km/s)^2 / M_sun
rho_pc3 = 1.5e-7            # In units of M_sun / pc^3
R_H_pc = R / pc_in_m
a_R_pc = a_R / pc_in_m

# ---------------- Sérsic 2D Profile AU for circular velocities ----------------

# 1. Define the 2D Sersic intensity profile function
def I_Sersic(r_pc, I_e, Re_pc, n, bn):
    r_pc = np.maximum(r_pc, 1e-9) # Ensure input works with arrays
    return I_e * np.exp(-bn * ((r_pc / Re_pc)**(1/n) - 1))

# 2. Calculate the intensity at the effective radius needed for total mass normalization
# Find I_e such that integrating I(r) gives the required M_total
# The integrated projected light is L_tot = 2*pi*n*Gamma(2n) * I_e * R_e^2 / b_n^(2n)
# The total mass is M_tot = L_tot * M/L_ratio
I_e_ser_normalized = 1.0 # place holder

# 3. Function to get projected surface *mass* density (Sigma, M_sun/pc^2 initially)
def Sigma_ser(r_pc):
    I_r = I_Sersic(r_pc, I_e_ser_normalized, R_e_pc, n_ser, bn)
    return I_r * mass_to_light_ratio_ser

# 4. The vectorized function to generate the orbital velocity profile
def ecv_Sersic_AU_2D(radii_ly_array):
    r_arr_ly = np.atleast_1d(radii_ly_array)
    r_arr_pc = r_arr_ly * ly_in_pc

    # --- Cumulative Mass Integration using 2D projection ---
    # The enclosed mass M(r) = Integral(2*pi*R * Sigma(R) * dR)
    r_grid = r_arr_pc # Use the input array as the grid
    Sigma_grid = Sigma_ser(r_grid) # Units M_sun/pc^2

    # Integrand for M(r) is 2*pi*r*Sigma(r) [Units M_sun/pc]
    integrand_mass = 2.0 * math.pi * r_grid * Sigma_grid
    dr = np.diff(r_grid)
    seg_mass = 0.5 * (integrand_mass[1:] + integrand_mass[:-1]) * dr
    M_r_grid_unnorm = np.concatenate(([0.0], np.cumsum(seg_mass)))

    # Normalize the enclosed mass profile to match M_spiral_solar
    # The last value of M_r_grid_unnorm is the total integrated mass in M_sun (unnormalized)
    normalization_factor = M_spiral_solar / M_r_grid_unnorm[-1]
    M_r_grid_norm = M_r_grid_unnorm * normalization_factor # Units M_sun

    # --- Calculate Vc^2 = G * M(r) / r ---
    # Convert M(r) to kg, r to meters, G in SI
    M_r_grid_kg = M_r_grid_norm * M_sun
    r_grid_m = r_grid * pc_in_m
    r_grid_m_safe = np.maximum(r_grid_m, 1e-9) # Avoid division by zero at center
    # Vc^2 units: (m^3 kg^-1 s^-2) * kg / m = m^2 s^-2
    Vc_sq_grid_SI = G * (M_r_grid_kg) / r_grid_m_safe
    Vc_m_s = np.sqrt(np.maximum(Vc_sq_grid_SI, 0.0))

    return Vc_m_s # returns m/s array

    # --- Equivalently and consistently ---
    #a_o = G * M_r_grid_kg / r_grid_m_safe**2
    #return (a_o * G * M_r_grid_kg)**0.25


def ecv_Total_AU_2D(radii_ly_array):
    r_arr_ly = np.atleast_1d(radii_ly_array)
    r_arr_pc = r_arr_ly * ly_in_pc

    # --- Cumulative Mass Integration using 2D projection ---
    # The enclosed mass M(r) = Integral(2*pi*R * Sigma(R) * dR)
    r_grid = r_arr_pc # Use the input array as the grid
    Sigma_grid = Sigma_ser(r_grid) # Units M_sun/pc^2

    # Integrand for M(r) is 2*pi*r*Sigma(r) [Units M_sun/pc]
    integrand_mass = 2.0 * math.pi * r_grid * Sigma_grid
    dr = np.diff(r_grid)
    seg_mass = 0.5 * (integrand_mass[1:] + integrand_mass[:-1]) * dr
    M_r_grid_unnorm = np.concatenate(([0.0], np.cumsum(seg_mass)))

    # Normalize the enclosed mass profile to match M_spiral_solar
    # The last value of M_r_grid_unnorm is the total integrated mass in M_sun (unnormalized)
    normalization_factor = M_spiral_solar / M_r_grid_unnorm[-1]
    M_r_grid_norm = M_r_grid_unnorm * normalization_factor # Units M_sun

    # --- Calculate Vc^2 = G * M(r) / r ---
    # Convert M(r) to kg, r to meters, G in SI
    M_r_grid_kg = M_r_grid_norm * M_sun
    r_grid_m = r_grid * pc_in_m
    r_grid_m_safe = np.maximum(r_grid_m, 1e-9) # Avoid division by zero at center

    # Local Vc^2 units: (m^3 kg^-1 s^-2) * kg / m = m^2 s^-2
    Vc_sq_grid_SI = G * M_r_grid_kg / r_grid_m_safe
    Vc_m_s = np.sqrt(np.maximum(Vc_sq_grid_SI, 0.0))

    # Total Vc^2 via generalized baryonic Tully-Fisher relation towards density balance
    rho_o_grid = np.maximum(3*M_r_grid_kg / (4 * math.pi * r_grid_m_safe**3),0.1*rho)
    f = np.maximum(0.0,1 - 0.1*(M_env/M_spiral_solar/M_sun)**(1/3)*rho / rho_o_grid)
    a_o = G * M_r_grid_kg / r_grid_m_safe**2
    M_R_grid = (4.0/3.0) * np.pi * rho * r_grid_m_safe**3
    a_R = c**2/(R-r_grid_m_safe)

    return (f*(a_o + a_R/(2*math.pi)) * G * (M_r_grid_kg))**0.25

# ---------------- Hernquist Model for velocity dispersion ---------------------

def M_enclosed(r_m, M, a):
    r = np.asarray(r_m, dtype=float)
    return M * (r**2) / (r + a)**2

def rho_hern(r_m, M, a):
    r = np.asarray(r_m, dtype=float)
    return (M * a) / (2 * np.pi * r * (r + a)**3)

def rho_uni(r_m):
    r = np.asarray(r_m, dtype=float)
    return c**2 / (4 * np.pi * G * (R - r)**2)

def evd_Hernquist_rho(r_ly, M=M_total, a=r_scale_m, r_max_factor=2000.0, n_grid=10000):
    scalar_input = np.isscalar(r_ly)
    r_arr_ly = np.atleast_1d(r_ly).astype(float)
    r_arr_m  = r_arr_ly * ly_in_m

    # --- dense radial grid for full curve ---
    r_min = max(1e-12 * a, 1.0)
    r_max = r_max_factor * a
    r_grid = np.logspace(np.log10(r_min), np.log10(r_max), n_grid)

    # --- cumulative integral ---
    integrand = G*M_enclosed(r_grid, M, a) / r_grid**2

    dr = np.diff(r_grid)
    seg = 0.5*(integrand[1:] + integrand[:-1])*dr
    cum = np.concatenate(([0.0], np.cumsum(seg)))
    total_integral = cum[-1]
    I_r_grid = total_integral - cum  # full σ² numerator

    # --- numerical σ ---
    sigma = np.sqrt(np.maximum(I_r_grid/(4*np.pi), 0.0))

    # --- interpolate to requested radii ---
    sigma_at_r = np.interp(r_arr_m, r_grid, sigma)

    # --- analytic central dispersion ---
    sigma0 = np.sqrt(G * M / (12 * a))
    #print("analytic  ",sigma0)
    #print("calculated",sigma_at_r[0])

    if scalar_input:
        return float(sigma_at_r[0])
    return sigma_at_r

def evd_Total_rho(r_ly, M=M_total, a=r_scale_m, r_max_factor=2000.0, n_grid=10000):

    scalar_input = np.isscalar(r_ly)
    r_arr_ly = np.atleast_1d(r_ly).astype(float)
    r_arr_m  = r_arr_ly * ly_in_m

    # --- dense radial grid for full curve ---
    r_min = max(1e-12 * a, 1.0)
    r_max = r_max_factor * a # r_max needs to be large enough initially to find our limit
    r_grid = np.logspace(np.log10(r_min), np.log10(r_max), n_grid)

    # Find the index corresponding to the environment density
    R_limit_index = np.argmin(np.abs(rho_hern(r_grid, M, a) - .1*(M_env/M)**(1/3)*rho_uni(r_grid)))
    r_limit_m = r_grid[R_limit_index]
    r_grid_limited = r_grid[:R_limit_index + 1]
    rho_hern_limited = rho_hern(r_grid_limited, M, a)
    a_R = c**2/(R-r_grid_limited)

    # The integrand for the limited range
    integrand_limited = G*M_enclosed(r_grid_limited, M, a) / r_grid_limited**2
    #integrand_limited = G*rho_hern(r_grid_limited, M, a)
    dr = np.diff(r_grid_limited)
    seg = 0.5*(integrand_limited[1:] + integrand_limited[:-1])*dr
    cum = np.concatenate(([0.0], np.cumsum(seg)))
    total_integral = cum[-1]
    I_r_grid = total_integral - cum

    M_integrand_limited = G*M_enclosed(r_grid_limited, M, a)/ r_grid_limited
    dr = np.diff(r_grid_limited)
    M_seg = 0.5*(M_integrand_limited[1:] + M_integrand_limited[:-1])*dr
    M_cum = np.concatenate(([0.0], np.cumsum(M_seg)))
    M_total_integral = M_cum[-1]
    M_I_r_grid = M_total_integral - M_cum

    R_integrand_limited = G*M_enclosed(r_grid_limited, M, a)/(4*np.pi*np.pi*r_grid_limited**3) # Kepler
    #R_integrand_limited = G*rho_hern(r_grid_limited, M, a)/(2*np.pi)  # Kepler

    dr = np.diff(r_grid_limited)
    R_seg = 0.5*(R_integrand_limited[1:] + R_integrand_limited[:-1])*dr
    R_cum = np.concatenate(([0.0], np.cumsum(R_seg)))
    R_total_integral = R_cum[-1]
    R_I_r_grid = R_total_integral - R_cum

    # --- numerical σ ---
    #f = np.maximum(0.0,1 - rho_uni(r_grid_limited) / rho_hern(r_grid_limited, M, a))
    #sigma = (f*(np.maximum(I_r_grid, 0.0) + a_R)/(4*np.pi)*np.maximum(M_I_r_grid, 0.0))**0.25
    sigma = ((np.maximum(R_I_r_grid, 0.0) + a_R)/(4*np.pi)*np.maximum(M_I_r_grid, 0.0))**0.25
    sigma[-1] = 0.0
    # --- interpolate to requested radii ---
    sigma_at_r = np.interp(r_arr_m, r_grid_limited, sigma)

    # --- analytic central dispersion for comparison ---
    sigma0 = np.sqrt(G * M / (12 * a))
    #print("analytic  ",sigma0)
    #print("calculated",sigma_at_r[0])

    return sigma_at_r

# ---------------- Compute Profiles --------------------------------------------
radii_ly = np.logspace(1, 9, 1000)  # 100 ly to 1000,000,000 ly
radii_pc = radii_ly * ly_in_pc

vd_Hernquist_cluster = evd_Hernquist_rho(radii_ly,M_cluster,r_cluster_m)
vd_Total_cluster = evd_Total_rho(radii_ly,M_cluster,r_cluster_m)
vd_Hernquist_galaxy = evd_Hernquist_rho(radii_ly,M_galaxy,r_galaxy_m)
vd_Total_galaxy = evd_Total_rho(radii_ly,M_galaxy,r_galaxy_m)
vd_Hernquist_dwarf = evd_Hernquist_rho(radii_ly,M_dwarf,r_dwarf_m)
vd_Total_dwarf = evd_Total_rho(radii_ly,M_dwarf,r_dwarf_m)

cv_Sersic = ecv_Sersic_AU_2D(radii_pc)
cv_Total = ecv_Total_AU_2D(radii_pc)

vd_Hernquist_cluster_km_s = np.array(vd_Hernquist_cluster) / 1000
vd_Total_cluster_km_s = np.array(vd_Total_cluster) / 1000
vd_Hernquist_galaxy_km_s = np.array(vd_Hernquist_galaxy) / 1000
vd_Total_galaxy_km_s = np.array(vd_Total_galaxy) / 1000
vd_Hernquist_dwarf_km_s = np.array(vd_Hernquist_dwarf) / 1000
vd_Total_dwarf_km_s = np.array(vd_Total_dwarf) / 1000
cv_Sersic_km_s = np.array(cv_Sersic) / 1000
cv_Total_km_s = np.array(cv_Total) / 1000


# -------------------- Plotting ------------------------------------------------

def plot_velocity_profiles(radii,
                           vd_Total_cluster_km_s, vd_Hernquist_cluster_km_s,
                           vd_Total_galaxy_km_s, vd_Hernquist_galaxy_km_s,
                           vd_Total_dwarf_km_s, vd_Hernquist_dwarf_km_s,
                           cv_Total_km_s, cv_Sersic_km_s,
                           xscale='log', xlim=None, ylim=None, filename=None, path=path,
                           yscale='linear'):
    fig, ax1 = plt.subplots(figsize=(8, 7))

    # Velocity dispersion
    ax1.plot(radii, vd_Total_cluster_km_s, '-', lw=4, color='blue', label=r"Cluster Total $\sigma$")
    ax1.plot(radii, vd_Hernquist_cluster_km_s, '-', lw=2, color='blue', label=r"Cluster Newton $\sigma$")
    ax1.plot(radii, vd_Total_galaxy_km_s, '-', lw=4, color='green', label=r"Galaxy Total $\sigma$")
    ax1.plot(radii, vd_Hernquist_galaxy_km_s, '-', lw=2, color='green', label=r"Galaxy Newton $\sigma$")
    ax1.plot(radii, vd_Total_dwarf_km_s, '-', lw=4, color='red', label=r"Dwarf Total $\sigma$")
    ax1.plot(radii, vd_Hernquist_dwarf_km_s, '-', lw=2, color='red', label=r"Dwarf Newton $\sigma$")

    # Circular velocity
    ax1.plot(radii, cv_Total_km_s, '-', lw=3, color='black', label="Spiral Total v")
    ax1.plot(radii, cv_Sersic_km_s, '-', lw=2, color='gray', label="Spiral Newton v")

    # Axis formatting
    ax1.set_xlabel(r"$r$ (light-years)", fontsize=20)
    ax1.set_ylabel(r"Velocity ($\sigma, v$) (km/s)", fontsize=20)

    # Set the axes scales
    ax1.set_xscale(xscale)
    ax1.set_yscale(yscale)

    if xlim:
        ax1.set_xlim(xlim)
    if ylim:
        ax1.set_ylim(ylim)
    ax1.tick_params(axis='both', which='major', labelsize=16)
    ax1.grid(True, which="both", ls="--", lw=0.5)

    ax1.legend(fontsize=12, loc='best')

    plt.tight_layout()
    if filename:
        plt.savefig(path + filename, dpi=300, bbox_inches='tight')
        print(f"Figure saved as: {filename}")
    plt.show()

"""
def plot_velocity_profiles(radii,
                           vd_Total_cluster_km_s, vd_Hernquist_cluster_km_s,
                           vd_Total_galaxy_km_s, vd_Hernquist_galaxy_km_s,
                           vd_Total_dwarf_km_s, vd_Hernquist_dwarf_km_s,
                           cv_Total_km_s, cv_Sersic_km_s,
                           xscale='log', xlim=None, ylim=None, filename=None, path=path,
                           yscale='linear'):
    fig, ax1 = plt.subplots(figsize=(8, 7))

    # --- Setup the second Y-axis (ax2) for circular velocity v_c ---
    ax2 = ax1.twinx()

    # --- Plotting Velocity Dispersion (on ax1, left axis) ---
    # Using the currently uncommented lines:
    ax1.plot(radii, vd_Total_galaxy_km_s, '--', lw=4, color='black', label=r"Galaxy Total $\sigma$")
    ax1.plot(radii, vd_Hernquist_galaxy_km_s, ':', lw=2, color='gray', label=r"Galaxy Newton $\sigma$")

    # To plot all sigmas using the commented out lines, uncomment below:
    # ax1.plot(radii, vd_Total_cluster_km_s, '--', lw=4, color='blue', label=r"Cluster Total $\sigma$")
    # ax1.plot(radii, vd_Hernquist_cluster_km_s, ':', lw=2, color='blue', label=r"Cluster Newton $\sigma$")
    # ax1.plot(radii, vd_Total_dwarf_km_s, '--', lw=4, color='red', label=r"Dwarf Total $\sigma$")
    # ax1.plot(radii, vd_Hernquist_dwarf_km_s, ':', lw=2, color='red', label=r"Dwarf Newton $\sigma$")

    # --- Plotting Circular Velocity (on ax2, right axis) ---
    ax2.plot(radii, cv_Total_km_s, '-', lw=3, color='black', label=r"Spiral Total $v_c$")
    ax2.plot(radii, cv_Sersic_km_s, '-', lw=2, color='gray', label=r"Spiral Newton $v_c$")

    # --- Axis Formatting ---

    # X-Axis configuration (shared by both axes)
    ax1.set_xlabel(r"$r$ (light-years)", fontsize=20)
    ax1.set_xscale(xscale)
    if xlim:
        ax1.set_xlim(xlim)
    ax1.tick_params(axis='x', which='major', labelsize=16)
    ax1.grid(True, which="both", ls="--", lw=0.5)

    # Y-Axis 1 (Left - Velocity Dispersion)
    ax1.set_ylabel(r"$\sigma$ (km/s)", fontsize=20, color='black')
    ax1.tick_params(axis='y', which='major', labelsize=16, colors='black')
    ax1.set_yscale(yscale) # Apply log or linear scale
    if ylim:
        # Scale the left Y-axis range if ylim is specified
        ax1.set_ylim(ylim)

    # Y-Axis 2 (Right - Circular Velocity)
    ax2.set_ylabel(r"$v$ (km/s)", fontsize=20, color='black')
    ax2.tick_params(axis='y', which='major', labelsize=16, colors='black')
    ax2.set_yscale(yscale) # Match the scale type (log/linear) of ax1

    # using a factor of 2
    if ylim:
        # If ax1 is (ymin, ymax), ax2 becomes (ymin*2, ymax*2) or similar scaling logic
        # A 1-to-1 scale is often preferred unless you need visual alignment:
        ax2.set_ylim(ylim)
        # If you specifically need one axis exactly double the other:
        ax2.set_ylim(np.array(ylim) * 2.0)

    # Combine legends
    lines_1, labels_1 = ax1.get_legend_handles_labels()
    lines_2, labels_2 = ax2.get_legend_handles_labels()
    #ax1.legend(lines_1 + lines_2, labels_1 + labels_2, fontsize=12, loc='best')

    plt.tight_layout()
    if filename:
        plt.savefig(path + filename, dpi=300, bbox_inches='tight')
        print(f"Figure saved as: {filename}")
    plt.show()
"""

# Usage
plot_velocity_profiles(
    radii=radii_ly,
    vd_Total_cluster_km_s=vd_Total_cluster_km_s,
    vd_Hernquist_cluster_km_s=vd_Hernquist_cluster_km_s,
    vd_Total_galaxy_km_s=vd_Total_galaxy_km_s,
    vd_Hernquist_galaxy_km_s=vd_Hernquist_galaxy_km_s,
    vd_Total_dwarf_km_s=vd_Total_dwarf_km_s,
    vd_Hernquist_dwarf_km_s=vd_Hernquist_dwarf_km_s,
    cv_Total_km_s=cv_Total_km_s,
    cv_Sersic_km_s=cv_Sersic_km_s,
    ylim=(1, 10000),
    xlim=(1e2, 1e9),
#    ylim=(1, 300),
#    xlim=(1e2, 3e7),
    yscale='log',
    xscale='log',
    filename="log_log_plot.jpg"
)

plot_velocity_profiles(
    radii=radii_ly,
    vd_Total_cluster_km_s=vd_Total_cluster_km_s,
    vd_Hernquist_cluster_km_s=vd_Hernquist_cluster_km_s,
    vd_Total_galaxy_km_s=vd_Total_galaxy_km_s,
    vd_Hernquist_galaxy_km_s=vd_Hernquist_galaxy_km_s,
    vd_Total_dwarf_km_s=vd_Total_dwarf_km_s,
    vd_Hernquist_dwarf_km_s=vd_Hernquist_dwarf_km_s,
    cv_Total_km_s=cv_Total_km_s,
    cv_Sersic_km_s=cv_Sersic_km_s,
    ylim=(1, 2500),
    xlim=(1e2, 1e9),
#    ylim=(1, 300),
#    xlim=(1e3, 3e7),
    xscale='log',
    filename="log_lin_plot.jpg"
)
plot_velocity_profiles(
    radii=radii_ly,
    vd_Total_cluster_km_s=vd_Total_cluster_km_s,
    vd_Hernquist_cluster_km_s=vd_Hernquist_cluster_km_s,
    vd_Total_galaxy_km_s=vd_Total_galaxy_km_s,
    vd_Hernquist_galaxy_km_s=vd_Hernquist_galaxy_km_s,
    vd_Total_dwarf_km_s=vd_Total_dwarf_km_s,
    vd_Hernquist_dwarf_km_s=vd_Hernquist_dwarf_km_s,
    cv_Total_km_s=cv_Total_km_s,
    cv_Sersic_km_s=cv_Sersic_km_s,
    ylim=(1, 2500),
    xlim=(0, 2000000),
    yscale='log',
    xscale='linear',
    filename="lin_log_plot.jpg"
)

plot_velocity_profiles(
    radii=radii_ly,
    vd_Total_cluster_km_s=vd_Total_cluster_km_s,
    vd_Hernquist_cluster_km_s=vd_Hernquist_cluster_km_s,
    vd_Total_galaxy_km_s=vd_Total_galaxy_km_s,
    vd_Hernquist_galaxy_km_s=vd_Hernquist_galaxy_km_s,
    vd_Total_dwarf_km_s=vd_Total_dwarf_km_s,
    vd_Hernquist_dwarf_km_s=vd_Hernquist_dwarf_km_s,
    cv_Total_km_s=cv_Total_km_s,
    cv_Sersic_km_s=cv_Sersic_km_s,
    ylim=(0, 2500),
    xlim=(0, 2000000),
#    ylim=(0, 300),
#    xlim=(0, 5e5),
    xscale='linear',
    filename="lin_lin_plot.jpg"
)


In [ ]:
#@title DENSITY & ACCELERATION
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gammaincinv
from scipy.optimize import minimize_scalar
from scipy.integrate import quad

# ---------------- Constants ---------------------------------------------------
G = 6.67430e-11              # Gravitational constant, m^3 kg^-1 s^-2
c = 299792458                # Speed of light, m/s
M_sun = 1.98847e30           # Solar mass, kg
ly_in_m = 9.461e15           # Light-year in meters
pc_in_m = 3.0857e16          # Parsec in meters
ly_in_pc = 0.306601          # Light-year in parsecs
pc_in_ly = 3.26156           # Parsec in light-years
t_universe = 13.8e9 * 365.25 * 24 * 3600  # Age of universe in seconds
a_R = c / t_universe         # Cosmic acceleration
rho = 1/(4*math.pi*G*t_universe**2)   # universal density kgm^-3

# ---------------- Galaxy Parameters -------------------------------------------
#M_total = 1e15 * M_sun     # Cluster
M_total = 1e11 * M_sun     # Galaxy
#M_total = 1e7 * M_sun      # Dwarf

# Hernquist (Elliptical Galaxy)
#r_scale_ly = 1e6           # Cluster
r_scale_ly = 10000         # Galaxy
#r_scale_ly = 1000          # Dwarf
r_scale_m = r_scale_ly * ly_in_m

# Sérsic (Spiral Galaxy)
M_spiral_solar = M_total/M_sun
R_e_ly = 12000
#R_e_ly = 1000          # Dwarf
R_e_m = R_e_ly * ly_in_m
R_e_pc = R_e_ly * ly_in_m / pc_in_m
R_e_pc2 = R_e_pc
n_ser = 1.0
bn = gammaincinv(2 * n_ser, 0.5)
pn = 1.0 - 0.6097 / n_ser + 0.05463 / n_ser**2
mass_to_light_ratio_ser = 1.0

G_AU = 4.302e-3 # G in units of pc * (km/s)^2 / M_sun
rho_pc3 = 1.5e-7 # In units of M_sun / pc^3
R = c * t_universe
R_H_pc = R / pc_in_m
a_R_pc = a_R / pc_in_m


# ---------------- Sérsic Profile ----------------------------------------------

# 1. Define the 2D Sersic intensity profile function (VECTORIZED using np.exp)
def I_Sersic(r_pc, I_e, Re_pc, n, bn):
    r_pc = np.atleast_1d(r_pc)
    return I_e * np.exp(-bn * ((r_pc / Re_pc)**(1/n) - 1))

I_e_ser_normalized = 1.0

# 2. Function to get projected surface mass density (VECTORIZED)
def Sigma_ser(r_pc):
    I_r = I_Sersic(r_pc, I_e_ser_normalized, R_e_pc, n_ser, bn)
    return I_r * mass_to_light_ratio_ser # Returns value proportional to Sigma (kg/m^2)

def unnormalized_rho_sersic_n2(r_in, Re):
    r = np.maximum(r_in, 1e-9)
    density_propto = (r / Re)**(-pn) * np.exp(-bn * (r / Re)**(1/2.))
    return density_propto

def get_normalized_sersic_density_func(M_total, Re_pc):
    # Calculate normalization constant for THIS galaxy's M and Re *once*
    def volume_integrand_unnorm(r_in, Re):
        return 4.0 * np.pi * r_in**2 * unnormalized_rho_sersic_n2(r_in, Re)

    max_radius = R_H_pc
    unnorm_mass, err = quad(volume_integrand_unnorm, 0, max_radius,
                        args=(Re_pc,),
                        points=[Re_pc], # Force integration to sample around the scale radius
                        epsrel=1e-5, epsabs=1e-5)
    C_norm = M_total / unnorm_mass

    # Return the callable normalized density function in M_sun/pc^3
    def rho_sersic_n2_normalized_au(r_in):
        return C_norm * unnormalized_rho_sersic_n2(r_in, Re_pc)

    return rho_sersic_n2_normalized_au

# The density function for M_spiral_solar and R_e_pc
_rho_sersic_func_au = get_normalized_sersic_density_func(M_spiral_solar, R_e_pc)


def rho_Sersic(r_ly):
    """
    Calculates the 3D Sersic density profile for the given radii in light-years.
    Returns density in SI units (kg/m^3).
    """
    r_ly = np.atleast_1d(r_ly)
    r_pc = r_ly * ly_in_pc # Convert input LY array to PC
    r_m = r_ly * ly_in_m   # Convert input LY array to meters

    # Get density in M_sun/pc^3
    rho_au = _rho_sersic_func_au(r_pc)

    # Convert density from M_sun/pc^3 to kg/m^3 (SI units)
    # (M_sun/pc^3) * (kg/M_sun) * (pc^3/m^3)
    rho_si = rho_au * M_sun / (pc_in_m**3)

    return rho_si

# Hernquist Density Profile
def rho_Hernquist(r_ly):
    r_m = np.atleast_1d(r_ly) * ly_in_m
    r_scale_m_safe = np.maximum(r_scale_m, 1e-9)
    r_m_safe = np.maximum(r_m, 1e-9)
    # This formula assumes M_total is the already normalized total mass in kg
    return (M_total * r_scale_m_safe) / (2 * np.pi * r_m_safe * (r_m_safe + r_scale_m_safe)**3)

# Function to get total enclosed mass M(r) from 2D Sigma profile
def M_Sersic(r_ly_array):
    r_m = np.atleast_1d(r_ly_array) * ly_in_m
    r_pc = r_m / pc_in_m

    # M(r) = Integral(2*pi*R * Sigma(R) * dR)
    Sigma_grid = Sigma_ser(r_pc) # Proportional units (kg/m^2)

    integrand_mass = 2.0 * math.pi * r_m * Sigma_grid # Units are proportional to kg/m
    dr = np.diff(r_m)
    seg_mass = 0.5 * (integrand_mass[1:] + integrand_mass[:-1]) * dr
    M_r_grid_unnorm = np.concatenate(([0.0], np.cumsum(seg_mass)))

    # Normalize the mass profile to match M_total
    normalization_factor = M_total / M_r_grid_unnorm[-1]
    M_r_grid_norm = M_r_grid_unnorm * normalization_factor # Units kg

    return M_r_grid_norm

# Hernquist Acceleration Profile
def acc_Hernquist(r_ly):
    r_m = np.atleast_1d(r_ly) * ly_in_m
    r_m_safe = np.maximum(r_m, 1e-9)
    # The Hernquist potential allows direct calculation of enclosed mass M_enc(r)
    M_enc = M_total * (r_m_safe / (r_m_safe + r_scale_m))**2
    return G * M_enc / r_m_safe**2

# Sersic Acceleration Profile
def acc_Sersic(r_ly):
    r_m = np.atleast_1d(r_ly) * ly_in_m
    r_m_safe = np.maximum(r_m, 1e-9)
    M_enc = M_Sersic(r_ly) # Calls vectorized mass function
    return G * M_enc / r_m_safe**2

# Universal Density Profile
def rho_uni(r_ly):
    r_m = np.atleast_1d(r_ly) * ly_in_m
    r_m_safe = np.maximum(r_m, 1e-9)
    return c**2 / (4 * np.pi * G * (R - r_m_safe)**2)

# Universal Acceleration Profile
def acc_uni(r_ly):
    r_m = np.atleast_1d(r_ly) * ly_in_m
    r_m_safe = np.maximum(r_m, 1e-9)
    return c**2/(R-r_m_safe)

# ---------------- Compute Density and Acceleration Profiles -------------------

radii_ly = np.logspace(1, 9, 1000) # Define radii once
radii_pc = radii_ly * ly_in_m

rho_Hernquist_vals = rho_Hernquist(radii_ly)
rho_Sersic_vals = rho_Sersic(radii_ly)
rho_uni_vals = rho_uni(radii_ly)
acc_Hernquist_vals = acc_Hernquist(radii_ly)
acc_Sersic_vals = acc_Sersic(radii_ly)
acc_uni_vals = acc_uni(radii_ly)

# ----------------- Plot Density and Acceleration Profiles ---------------------

def plot_density_acceleration(radii, rho_hern, rho_sersic, rho_uni, acc_hern, acc_sersic, acc_uni,
                               xscale='log', xlim=None, ylim_rho=None, ylim_acc=None):
    fig, ax1 = plt.subplots(figsize=(8, 7))

    # Density (left axis)
    ax1.plot(radii, rho_hern, lw=2, color='blue', label="Hernquist density")
    ax1.plot(radii, rho_sersic, lw=2, color='green', label="Sérsic density (Approx 3D)")
    ax1.plot(radii, rho_uni, lw=4, color='black', label="Universal density")
    #ax1.axhline(y=rho, color='black', lw=4, label="Universal density")

    ax1.set_xlabel("$r$ (light-years)", fontsize=20)
    ax1.set_ylabel("Density (kg/m³)", color='blue', fontsize=20)
    ax1.set_xscale(xscale)
    ax1.set_yscale('log')
    if xlim:
        ax1.set_xlim(xlim)
    if ylim_rho:
        ax1.set_ylim(ylim_rho)
    ax1.tick_params(axis='y', which='major', labelsize=16, colors='blue')
    ax1.tick_params(axis='x', which='major', labelsize=16, colors='black')
    ax1.grid(True)

    # Acceleration (right axis)
    ax2 = ax1.twinx()
    ax2.plot(radii, acc_hern, '--', lw=2, color='red', label="Hernquist acceleration")
    ax2.plot(radii, acc_sersic, '--', lw=2, color='orange', label="Sérsic acceleration (2D integrated)")
    ax2.plot(radii, acc_uni, '--', lw=4, color='black', label="Universal acceleration")

    ax2.set_ylabel("Acceleration (m/s²)", color='red', fontsize=20)
    ax2.set_xscale(xscale)
    ax2.set_yscale('log')
    if ylim_acc:
        ax2.set_ylim(ylim_acc)
    ax2.tick_params(axis='both', which='major', labelsize=16, colors='red')

    # Combine legends
    lines_1, labels_1 = ax1.get_legend_handles_labels()
    lines_2, labels_2 = ax2.get_legend_handles_labels()
    ax1.legend(lines_1 + lines_2, labels_1 + labels_2, fontsize=12, loc='lower left')

    plt.tight_layout()
    plt.show()

# Usage
plot_density_acceleration(
    radii=radii_ly,
    rho_hern=rho_Hernquist_vals,
    rho_sersic=rho_Sersic_vals,
    rho_uni=rho_uni_vals,
    acc_hern=acc_Hernquist_vals,
    acc_sersic=acc_Sersic_vals,
    acc_uni=acc_uni_vals,
    xscale='log',
    ylim_rho=(1e-30, 1e-16),
    ylim_acc=(1e-10, 1e-8),
    xlim=(1e1, 1e7)
)


In [ ]:
#@title FLUX BALANCE BLOW-UP
# Plot overdensity falloff near flux balance

import numpy as np
import math
import matplotlib.pyplot as plt

# ---------------- Constants ---------------------------------------------------
G = 6.67430e-11              # Gravitational constant, m^3 kg^-1 s^-2
c = 299792458                # Speed of light, m/s
M_sun = 1.98847e30           # Solar mass, kg
ly_in_m = 9.461e15           # Light-year in meters
pc_in_m = 3.0857e16          # Parsec in meters
t_universe = 13.8e9 * 365.25 * 24 * 3600  # Age of universe in seconds
a_R = c / t_universe         # Cosmic acceleration
rho = 1/(4*math.pi*G*t_universe**2)   # universal density kgm^-3
#rho = c**2*c*t_universe/(4*math.pi*G*(c*t_universe)**3)

# ---------------- Galaxy Parameters -------------------------------------------
M_total = 1e11 * M_sun       # Total galaxy mass

# Compute characteristic radius
r_c_m = (M_total / rho)**(1/3)  # in meters
r_c_ly = r_c_m / ly_in_m  # convert meters to light-years

# Define radius range (slightly below and above r_c)
radii_ly = np.linspace(0.0*r_c_ly, 5.0*r_c_ly, 1000)
radii_r_by_rc = radii_ly / r_c_ly  # dimensionless r/r_c

# ---------------- Define Functions --------------------------------------------

# Exact cubic model
def delta_rho_exact(r_by_rc):
    return 1 - r_by_rc**3

# Logistic approximation
def delta_rho_logistic(r_by_rc):
    return 1 / (1 + r_by_rc**3)**2

# Cutoff power law approximation
def delta_rho_cutoff(r_by_rc, n=3):
    return 1 / (1 + r_by_rc**n)

# ---------------- Evaluate ----------------------------------------------------
delta_exact = delta_rho_exact(radii_r_by_rc)
delta_logistic = delta_rho_logistic(radii_r_by_rc)
delta_cutoff = delta_rho_cutoff(radii_r_by_rc, n=3)

# ---------------- Plotting ----------------------------------------------------
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 10), sharex=False)

# Linear scale plot
ax1.plot(radii_r_by_rc, delta_exact, label=r'Exact: $1 - (r/r_{fb})^3$', lw=2)
ax1.plot(radii_r_by_rc, delta_logistic, '--', label=r'Logistic: $1/(1+(r/r_{fb})^3)^2$', lw=2)
ax1.plot(radii_r_by_rc, delta_cutoff, ':', label=r'Cutoff power law: $1/(1+(r/r_{fb})^3)$', lw=2)
ax1.axvline(1, color='gray',  lw=3, linestyle='--', label=r'$r=r_{fb}$')
ax1.axhline(0, color='gray', lw=3, linestyle='--', label=r'$\delta_\rho=0$')
ax1.set_xlabel(r'$r/r_{fb}$', fontsize=12)
ax1.set_ylabel(r'$\delta_\rho$', fontsize=12)
ax1.set_title(r'Overdensity $\delta_\rho$ vs. $r/r_{fb}$', fontsize=14)
ax1.legend()
ax1.grid(True)
ax1.set_ylim(-1, 1.2)
ax1.set_xlim(0, 2)

# Log-x scale plot
ax2.plot(radii_r_by_rc, delta_exact, label=r'Exact: $1 - (r/r_{fb})^3$', lw=2)
ax2.plot(radii_r_by_rc, delta_logistic, '--', label=r'Logistic: $1/(1+(r/r_{fb})^3)^2$', lw=2)
ax2.plot(radii_r_by_rc, delta_cutoff, ':', label=r'Cutoff power law: $1/(1+(r/r_{fb})^3)$', lw=2)
ax2.axvline(1, color='gray', lw=3, linestyle='--', label=r'$r=r_{fb}$')
ax2.axhline(0, color='gray', lw=3, linestyle='--', label=r'$\delta_\rho=0$')
ax2.set_xscale('log')
ax2.set_xlabel(r'$r/r_{fb}$ (log scale)', fontsize=12)
ax2.set_ylabel(r'$\delta_\rho$', fontsize=12)
ax2.set_title(r'Overdensity $\delta_\rho$ vs. $r/r_{fb}$', fontsize=14)
ax2.legend()
ax2.grid(True)
ax2.set_ylim(-1.1, 1.2)
ax2.set_xlim(0.1, 10)

plt.tight_layout()
plt.show()


In [ ]:
#@title DENSITIES ACROSS COSMOS
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator

plt.rcParams['font.family'] = 'DejaVu Serif'

# ---------------- Constants ---------------------------------------------------
G = 6.67430e-11                 # Gravitational constant, m^3 kg^-1 s^-2
c = 299792458                   # Speed of light, m/s
year = 365.25*24*3600 # 3.154e7 # seconds in a year
lightyear =  9.461e15           # Light-year in meters
age = 13800000000*365.25*24*3600

rho = np.logspace(-27, 19, 500) # Density range: from 1e-30 to 1e20 kg/m^3
rho_now = 6.28663585785629E-27
t = np.sqrt(1 / (4 * np.pi * G * rho)) / year  # time in years
r = np.sqrt(c**2 / (4 * np.pi * G * rho)) / lightyear  # distance in lightyears

# Characteristic densities (approximate values in kg/m^3)
objects = {
    'Stellar black hole': 1e18,           # ~10 M☉ within ~30 km (Schwarzschild radius)
    'Neutron star': 1e17,                 # Typical 1.4 M☉ in ~10 km radius
    'White dwarf': 1e9,                   # E.g., Sirius B (~1 M☉ in Earth-sized radius)
    'Sun': 1.41e3,                        # Mean solar density (reference star)
    'Red dwarf': 1e2,                     # Red dwarfs (e.g., Proxima Centauri)
    'Giant star': 1e-5,                   # E.g., Betelgeuse Much lower average density due to size
    'Globular cluster': 1e-17,            # E.g., Omega Centauri
    'Dwarf galaxy': 1e-20,                # E.g., Large Magellanic Cloud
    'Spiral galaxy': 1e-21,               # E.g., Milky Way average
    'Elliptical galaxy': 1e-22,           # E.g., M87
    'Ultra-diffuse galaxy': 1e-23,        # E.g., Dragonfly 44
    'Galaxy group': 1e-24,                # E.g., Local Group
    'Galaxy cluster': 1e-25,              # E.g., Virgo Cluster
    'Supercluster': 1e-26,                # E.g., Laniakea Supercluster
#    'Voids': 1e-27,                       #
}

# Density range: from 1e-30 to 1e20 kg/m^3
rho = np.logspace(-27, 19, 500)
rho_now = 6.28663585785629E-27

# Time and radius
t = np.sqrt(1 / (4 * np.pi * G * rho)) / year  # time in years
r = c*t / lightyear  # radius in lightyears

rho_points = np.array(list(objects.values()))
t_points = np.sqrt(1 / (4 * np.pi * G * rho_points)) / year

# ---------------- Plotting ----------------------------------------------------
fig, ax1 = plt.subplots(figsize=(8, 6))

# Left y-axis: time
ax1.loglog(rho, t, label=r"$t = \left(\frac{1}{4\pi G \rho}\right)^{1/2}$", linewidth=1, color='gray')
ax1.scatter(rho_points, t_points, color='white', edgecolor='black', linewidth=1, s=100, zorder=5)

# Annotations
for name, x, y in zip(objects.keys(), rho_points, t_points):
    ax1.annotate(name, (x, y), textcoords="offset points", xytext=(8,-5), ha='left', fontsize=7.5)

# Axis labels
ax1.set_xlabel(r"$\rho$ (kg/m³)", fontsize=16)
ax1.set_ylabel(r"$t$ (years)", fontsize=16)
ax1.tick_params(axis='both', labelsize=12)
ax1.set_xlim(1e-33, 1e19)
ax1.set_ylim(1e-15, 1e12)
ax1.invert_xaxis()  # Reverse the x-axis
ax1.xaxis.set_major_locator(LogLocator(base=10.0, numticks=10))
ax1.yaxis.set_major_locator(LogLocator(base=10.0, numticks=10))
ax1.grid(True, which="both", ls="--", lw=0.5)

plt.tight_layout()
plt.savefig("plot.png", dpi=300, bbox_inches='tight')
plt.show()
from google.colab import files
files.download("plot.png")